In [22]:
import numpy as np
import pandas as pd
from pygmid import Lookup as lk

#### Technology data

In [23]:
nfet_01v8_lvt = lk('./simulation/nfet_01v8_lvt.mat')

#### Specifications

In [24]:
gm = 10e-3

#### Design choices

In [25]:
l = 0.15
gm_id = np.array([6, 20])

#### Sizing and benchmarking

In [26]:
id = gm/gm_id
jd = nfet_01v8_lvt.lookup('ID_W', GM_ID=gm_id, L=l)
w = id/jd

cgg_w = nfet_01v8_lvt.lookup('CGG_W', GM_ID=gm_id, L=l)
cgg = w*cgg_w
ft = gm/cgg/2/np.pi
cgd_w = nfet_01v8_lvt.lookup('CGD_W', GM_ID=gm_id, L=l)
cgd = w*cgd_w
cgg_intr = cgg-2*cgd

df = pd.DataFrame([gm_id, id, w, cgg, ft, cgg_intr], ['gm_id', 'id', 'w', 'cgg', 'ft', 'cgg_intr'], columns=['option1', 'option2']); df

,option1,option2
gm_id,6.000000e+00,2.000000e+01
id,1.666667e-03,5.000000e-04
w,2.347374e+01,1.624752e+02
cgg,2.844918e-14,1.790295e-13
ft,5.594359e+10,8.889872e+09
cgg_intr,1.721516e-14,1.012724e-13


#### Spice validation

In [27]:
%%writefile ./simulation/sizing_diffpair.spice
** differential pair sizing example

.include /foss/pdks/sky130A/libs.tech/ngspice/corners/tt.spice
.param mc_mm_switch=0
.param lx=0.15 wx=23.47 nfx=1 idx=1.6666m
.save @m.xm1a.msky130_fd_pr__nfet_01v8_lvt

XM1a d g s 0 sky130_fd_pr__nfet_01v8_lvt L={lx} W={wx} nf={nfx} ad='int((nf+1)/2) * W/nf * 0.29' as='int((nf+2)/2) * W/nf * 0.29' pd='2*int((nf+1)/2) * (W/nf + 0.29)'
+ ps='2*int((nf+2)/2) * (W/nf + 0.29)' nrd='0.29 / W' nrs='0.29 / W' sa=0 sb=0 sd=0 mult=1 m=1
XM1b d g s 0 sky130_fd_pr__nfet_01v8_lvt L={lx} W={wx} nf={nfx} ad='int((nf+1)/2) * W/nf * 0.29' as='int((nf+2)/2) * W/nf * 0.29' pd='2*int((nf+1)/2) * (W/nf + 0.29)'
+ ps='2*int((nf+2)/2) * (W/nf + 0.29)' nrd='0.29 / W' nrs='0.29 / W' sa=0 sb=0 sd=0 mult=1 m=1
vg  g  0  1
vd  d  0  1
is  s  0  {2*idx}

.control
  op
  *show
  print @m.xm1a.msky130_fd_pr__nfet_01v8_lvt[gm]
  print @m.xm1a.msky130_fd_pr__nfet_01v8_lvt[cgg]
.endc
.end

Overwriting ./simulation/sizing_diffpair.spice


In [28]:
!/foss/tools/bin/ngspice -b ./simulation/sizing_diffpair.spice


Note: Compatibility modes selected: hs a



Circuit: ** differential pair sizing example

option SCALE: Scale is set to 1e-06 for instance and model parameters
Doing analysis at TEMP = 27.000000 and TNOM = 27.000000

Using SPARSE 1.3 as Direct Linear Solver
 Reference value :  0.00000e+00
No. of Data Rows : 1
@m.xm1a.msky130_fd_pr__nfet_01v8_lvt[gm] = 1.016733e-02
@m.xm1a.msky130_fd_pr__nfet_01v8_lvt[cgg] = 1.731567e-14
Note: Simulation executed from .control section 
